# Convert SLAMM raster outputs to polygons for analyses

In [1]:
import geopandas as gpd
import pandas as pd
import rioxarray as rx
import geowombat as gw
from rasterio.features import shapes
import numpy as np

c:\Users\AMarley.ERG\.conda\envs\geospatial-new\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# data paths
RAW_DATA = "../data/raw/"
CLEAN_DATA = "../data/clean/"
SLAMM_DATA = "C:/Users/AMarley.ERG/OneDrive - Eastern Research Group/Projects/snep_saltmarsh/SLAMM_runs/"

cc1_data = SLAMM_DATA + "cc_1/"
cc1_output = "../output/cc1/"

ma2_data = SLAMM_DATA + "ma_2/"
ma2_output = "../output/ma2/"

ri2_data = SLAMM_DATA + "ri2/"
ri2_output = "../output/ri2/"

Polygonize the raster

In [3]:
# Make inundation frequency key to add to dataframe
inun_freq_key_list = [
    [0, 'always inundated'], 
    [1, '30-day inundation'], 
    [2, '60-day inundation'], 
    [3, '90-day inundation'], 
    [4, '10-year storm'], 
    [5, '100-year storm'], 
    [8, 'protected by dikes']
]

inun_freq_key_df = pd.DataFrame(inun_freq_key_list, columns=['inun_freq_code', 'inun_freq'])

In [4]:
slamm_key = pd.read_csv(RAW_DATA + 'slamm_cats.csv')
slamm_key.head()

,Category Name,SLAMM Number,Open Water,Tidal,NonTidal Wet.,Dry Land,Developed,Aggregation Category
0,Developed Dry Land,1,0,0,0,1,1,Aggregated Non Tidal
1,Undeveloped Dry Land,2,0,0,0,1,0,Aggregated Non Tidal
2,Swamp,3,0,0,1,0,0,Freshwater Non-Tidal
3,Cypress Swamp,4,0,0,1,0,0,Freshwater Non-Tidal
4,Inland-Fresh Marsh,5,0,0,1,0,0,Freshwater Non-Tidal


In [5]:
# Convert Asc to Tif for easier processing
def conv_to_tif(asc_path: str, which_crs, tif_path: str, is_flood_data: True):

    # Open ASCI file
    asc_file = rx.open_rasterio(asc_path)

    # set the crs
    # for MA marshes it's: WGS 84 / UTM zone 19N
    asc_file.rio.write_crs(which_crs, inplace=True)

    if is_flood_data:
        # remove unnecessary values
        asc_file = asc_file.where(asc_file>=0)
        asc_file = asc_file.where(asc_file!=6)
        asc_file = asc_file.where(asc_file!=7)

    # wrtie to tif
    asc_file.rio.to_raster(tif_path)

In [7]:
def tif_to_polygon(rast_file_path:str, category_key:pd.DataFrame, key_col_name:str, which_year:str, marsh:str, outputpath:str, is_flood_data: True):

    # read in raster and convert the float type
    rast_file = rx.open_rasterio(rast_file_path)
    rast_file = rast_file.astype("float32")

    # Convert the raster data to a numpy array
    mask = rast_file.data[0] != rast_file.rio.nodata
    results = (
        {'properties': {key_col_name: v}, 'geometry': s}
        for i, (s, v) in enumerate(
            shapes(rast_file.data[0], mask=mask, transform=rast_file.rio.transform())
        )
    )

    # Create a GeoDataFrame from the results
    results_gdf = gpd.GeoDataFrame.from_features(results, crs=rast_file.rio.crs)

    # Dissolve the geometry to simplify
    results_gdf = results_gdf.dissolve(by=key_col_name).reset_index()

    # remove NAs
    results_gdf = results_gdf[~results_gdf[key_col_name].isna()]

    if is_flood_data:
        # add inundation frequency descriptions
        results_gdf = results_gdf.merge(category_key, on=key_col_name, how='left')

    # add descriptor columns
    results_gdf[['year']] = which_year
    results_gdf[['marsh']] = marsh

    # save file
    results_gdf.to_file(outputpath)

    return results_gdf


In [8]:
def polygonize_raster(
        asc_path: str,
        which_crs: int,
        category_key:pd.DataFrame,
        key_col_name:str,
        which_year:str,
        marsh:str,
        outputpath:str,
        is_flood_data: bool = True):
    """_summary_

    Args:
        asc_path (str): path to the SLAMM output asc file
        which_crs (int): crs of the file
        category_key (pd.DataFrame): dataframe key to add final names onto the numeric categories
        key_col_name (str): name of column to join the key dataframe with
        which_year (str): what year is this associated with
        marsh (str): is a marsh present
        outputpath (str): _description_
        is_flood_data (bool, optional): is this inundation data. Defaults to True.
    """

    tif_path = asc_path[:-4] + ".tif"

    # first convert asc to tif file
    conv_to_tif(
        asc_path=asc_path, 
        which_crs=which_crs, 
        tif_path=tif_path,
        is_flood_data=is_flood_data
        )
    
    # polygonize the raster
    tif_to_polygon(
        rast_file_path=tif_path,
        category_key=category_key, 
        key_col_name=key_col_name, 
        which_year=which_year, 
        marsh=marsh, 
        outputpath=outputpath,
        is_flood_data=is_flood_data
    )


In [ ]:
polygonize_raster(
        asc_path=SLAMM_DATA + "ri2/with_marsh/1slr/ri2_dem_OUT, 2040, prov_med_1slr_00_base _GIS.ASC",
        which_crs=26918,
        category_key=inun_freq_key_df,
        key_col_name='SLAMMCODE',
        which_year=2040,
        marsh='marsh present',
        outputpath=CLEAN_DATA + 'slamm/ri2/veg/ri2_1slr_40_sm_veg.geojson',
        is_flood_data=False
    )

In [19]:
import glob
import re
from tqdm.auto import tqdm
FILE_PATH_PATTERN = re.compile(r"(cc_1|ma_2|ri2)/with_marsh/(1|2)slr/.*20(20|70|40).*_00_baseline _GIS.ASC", re.IGNORECASE)
pbar = tqdm(glob.glob(f"{SLAMM_DATA}/*/*/*/*.ASC"))
for fpath in pbar:
    fpath = fpath.replace("\\", "/")
    pbar.set_description(f"Processing {fpath.split('/')[-1]}")
    match = FILE_PATH_PATTERN.search(fpath)
    if not match:
        continue

    marsh_id = match.group(1).replace("_", "")
    slr_scenario = match.group(2)
    year_id = int(match.group(3))

    if marsh_id == 'ri2':
        crs=26918
    else:
        crs=3857

    polygonize_raster(
        asc_path=fpath,
        which_crs=crs,
        category_key=inun_freq_key_df,
        key_col_name='SLAMMCODE',
        which_year=2000 + year_id,
        marsh='no marsh',
        outputpath=CLEAN_DATA + f'slamm/{marsh_id}/veg/{marsh_id}_{slr_scenario}slr_{year_id}_sm_veg.geojson',
        is_flood_data=False
    )

  0%|          | 0/120 [00:00<?, ?it/s]

Processing ri2_dem_OUT, Initial Condition _Inund_Freq_GIS.ASC: 100%|██████████| 120/120 [12:12<00:00,  6.11s/it]            
